# Kr SiPM filter

First runs with the detector have several problems in data adquisition. The SiPMs electronics saturate when an alpha is produced in the detector and the control software must be restarted, which takes about 1 second. This causes some events to be incomplete. There are two similar scenarios:
1.- When the first event is taken, the SiPMs are saturated and the SiPM data is not stored for all the events in the same file.
2.- If the first event contains SiPM data, it is not guaranteed that all SiPM channels are alive (not saturated). In this case the sofware fills the dead channels' waveforms with 2\*\*16 - 1.

We must filter out these kind of events and keep only those that are in good shape for an analysis. More over, we take this oportunity to merge the (filtered) events into a single file.

In [ ]:
from __future__ import print_function
import sys
import glob
import tables as tb
import numpy as np

In [ ]:
def create_new_file(h5out, h5in, nfiles):
    NEVT, NPMT, PMTWL = h5in.root.RD.pmtrwf.shape
    NEVT, NSIPM, SIPMWL = h5in.root.RD.sipmrwf.shape
    NEVT *= nfiles

    
    rungroup = h5out.create_group(h5out.root, "Run")
    h5in.root.Run.runInfo.copy(newparent=rungroup)
    evt_out = h5out.create_earray(h5out.root.Run, "event_number",
                                  atom=tb.Int16Atom(),
                                  shape=(0,),
                                  expectedrows=NEVT)
    
    detgroup = h5out.create_group(h5out.root, "Detector")
    h5in.root.Detector.DetectorGeometry.copy(newparent=detgroup)

    snsgroup = h5out.create_group(h5out.root, "Sensors")
    h5in.root.Sensors.DataPMT.copy(newparent=snsgroup)
    h5in.root.Sensors.DataBLR.copy(newparent=snsgroup)
    h5in.root.Sensors.DataSiPM.copy(newparent=snsgroup)
    
    h5out.create_group(h5out.root,"RD")
    pmt_out = h5out.create_earray(h5out.root.RD, "pmtrwf",
                                  atom=tb.Int16Atom(),
                                  shape=(0, NPMT, PMTWL),
                                  expectedrows=NEVT)

    blr_out = h5out.create_earray(h5out.root.RD, "pmtblr",
                                  atom=tb.Int16Atom(),
                                  shape=(0, NPMT, PMTWL),
                                  expectedrows=NEVT)

    sipm_out = h5out.create_earray(h5out.root.RD, "sipmrwf",
                                   atom=tb.Int16Atom(),
                                   shape=(0, NSIPM, SIPMWL),
                                   expectedrows=NEVT)
    return evt_out, pmt_out, blr_out, sipm_out
    
def filter_events(h5in, fill_number = 2**16 - 1):
    if "/RD/sipmrwf" not in h5in:
        return np.array([])
    else:
        return np.arange(h5in.root.RD.sipmrwf.shape[0])
    """
    sipmrwf = h5in.root.RD.sipmrwf
    print([fill_number in wf for wf in sipmrwf])
    valid_events = [i for i, wf in enumerate(sipmrwf) if fill_number not in wf]
    return valid_events
    """

def Kr_sipm_filter(outputfilename, *inputfilenames):
    with tb.open_file(outputfilename, "w", filters=tb.Filters(complevel=4, complib="zlib")) as h5out:
        create_file = True
        n_events_in = 0
        n_events_out = 0
        for i, filename in enumerate(inputfilenames):
            print("Opening",filename)
            with tb.open_file(filename, "r") as h5in:
                filtered_events = filter_events(h5in)
                n_events_in += h5in.root.RD.pmtrwf.shape[0]
                n_events_out += len(filtered_events)
                if create_file and filtered_events.size:
                    evt_out, pmt_out, blr_out, sipm_out = create_new_file(h5out, h5in, len(inputfilenames))
                    create_file = False
                for evt in filtered_events:
                    evt_out.append(h5in.root.Run.event_number[evt][np.newaxis])
                    pmt_out.append(h5in.root.RD.pmtrwf[evt][np.newaxis])
                    blr_out.append(h5in.root.RD.pmtblr[evt][np.newaxis])
                    sipm_out.append(h5in.root.RD.sipmrwf[evt][np.newaxis])
        pmt_out.flush()
        blr_out.flush()
        sipm_out.flush()
    print("# events in = {}, # events out = {}, ratio = {}".format(n_events_in, n_events_out, n_events_out*1.0/n_events_in))

In [ ]:
#f.close()
f=tb.open_file("./../../Shifts/data/merged.h5")

In [ ]:
c = [ 2**16-1 in f.root.RD.sipmrwf[i] for i in range(2888) ]
print("Number of events with 2**16-1", sum( map( int, c ) ))
c = [ -1 in f.root.RD.sipmrwf[i] for i in range(2888) ]
print("Number of events with -1", sum( map( int, c ) ))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import mplFunctions as mpl
import sensorFunctions as sns

sipmdf = sns.read_data_sensors(f.root.Sensors.DataSiPM)

In [ ]:
sipmrwf = f.root.RD.sipmrwf
evt=0
maxsipm = np.unravel_index(sipmrwf[evt].argmax(), sipmrwf[evt].shape)[0]
print("SiPM with greatest peak is at index {} with ID {}".format(maxsipm, sipmdf.ix[maxsipm].channel))

#plt.plot(sipmrwf[evt, maxsipm, :])
#plt.plot(sipmrwf[evt, 800, :])
plt.plot(sipmrwf[evt].T)
plt.xlabel("time ($\mu$s)")
plt.ylabel("Energy (adc)")